# python环境配置

本文的内容框架主要参考了[Setting Up a Python Development Environment with and without Docker](https://nickjanetakis.com/blog/setting-up-a-python-development-environment-with-and-without-docker)，后面每节又各有参考（详见下文）。

写程序一个很重要的内容就是 setting up your computer 。什么意思呢？总的来说，包括以下几个部分：

1. What code editor should I use? 用什么编辑器
2. How do I install Python? 如何安装Python
3. How do I install my app’s dependencies? 如何安装程序依赖包
4. How do I install required external services? 如何安装必须的外部服务
5. How do I run my application? 如何运行自己的程序

## What Code Editor Should I Use?

python中比较常用的IDE有pycharm和vscode，随意选择，都可以。

## Install Python

首先安装python，这部分其实比较简单，google或者baidu下，应该很容易找到答案。注意和自己的操作系统匹配，如果是linux注意disto也要匹配。另外2020年，python2就不更新了，所以能用python3还是用python3。

## Getting Your Python App Running without Docker

关于docker是什么可以暂时不用管，因为这一节是不会用到它的。

接下来，就是如何安装所需程序包。有app’s dependencies，比如numpy，pandas等，也有 required external services 比如你的数据库。

先说如何安装依赖工具包。当然你可以使用pip或者conda一个个地安装，也可以使用如下代码更新pip安装的库。

In [ ]:
# pip批量更新
import pip
from subprocess import call
from pip._internal.utils.misc import get_installed_distributions

n = 1
s = len(get_installed_distributions())
for dist in get_installed_distributions():
    call("pip install --upgrade " + dist.project_name, shell=True)
    print("共有{}个库,正在更新第{}个库,请耐心等待.......".format(s, n))
    n += 1
print("{}个库全部更新完毕".format(s))

但是这显然是比较慢的，最关键的是，可能会有两个不同项目使用同一个包不同版本的情况，那就很麻烦了。所以就有人开发了一个叫做 Virtual Environment 的东西--Virtualenv，这个python工具可以为每个项目创建一个虚拟环境，这样项目互相之间就不会影响了。接下来记录下Virtual Environment及其相关的内容。

### Virtual Environment

在开发Python应用程序的时候，系统安装的Python3只有一个版本。所有第三方的包都会被pip安装到Python3的site-packages目录下。

如果我们要同时开发多个应用程序，那这些应用程序都会共用一个Python，就是安装在系统的Python 3。如果应用A需要jinja 2.7，而应用B需要jinja 2.6怎么办？

这种情况下，**每个应用可能需要各自拥有一套“独立”的Python运行环境**。**virtualenv**就是用来为一个应用**创建一套“隔离”的Python运行环境**。

首先，我们用pip安装virtualenv：

``` bash
pip install virtualenv
```

然后，假定我们要开发一个新的项目，需要一套独立的Python运行环境，可以这么做：

第一步，创建目录：

```bash
mkdir myproject
cd myproject
```

第二步，创建一个独立的Python运行环境，命名为venv：

```bash
virtualenv --no-site-packages venv
```

命令virtualenv就可以创建一个独立的Python运行环境，我们还加上了**参数--no-site-packages**，这样，**已经安装到系统Python环境中的所有第三方包都不会复制过来**，这样，我们就得到了一个不带任何第三方包的“干净”的Python运行环境。

新建的Python环境被放到当前目录下的venv目录。有了venv这个Python环境，可以用source进入该环境：

```bash
source venv/bin/activate
```

可以观察命令行，注意到命令提示符变了，**有个(venv)前缀**，表示当前环境是一个名为venv的Python环境。

然后可以正常安装各种第三方包，并运行python命令：

``` bash
pip install jinja2
```

在venv环境下，用pip安装的包都被安装到venv这个环境下，系统Python环境不受任何影响。也就是说，venv环境是专门针对myproject这个应用创建的。

退出当前的venv环境，使用deactivate命令：

```bash
deactivate 
```

此时就回到了正常的环境，现在pip或python均是在系统Python环境下执行。

完全**可以针对每个应用创建独立的Python运行环境**，这样就可以对每个应用的Python环境进行隔离。

virtualenv是如何创建“独立”的Python运行环境的呢？原理很简单，就是把系统Python复制一份到virtualenv的环境，用命令source venv/bin/activate进入一个virtualenv环境时，virtualenv会修改相关环境变量，让命令python和pip均指向当前的virtualenv环境。

### requirements.txt

说道配置环境，就要提到requirements.txt，接下来的内容参考：[What is the python requirements.txt?](https://www.idkrtm.com/what-is-the-python-requirements-txt/)

如果浏览github上的python项目，经常会看到它。它就是一个指定运行项目所需的python packages 的。一般是在项目根目录下。一个requirements.txt形如：

pyOpenSSL==0.13.1

pyparsing==2.0.1

python-dateutil==1.5

每行对应一个package，然后是它的版本号。版本号也很重要，毕竟版本变化之后，容易出bug。

现在考虑一个问题，如果有了requirements.txt，如何安装对应库。首先，看看freeze。

In [1]:
!pip freeze

absl-py==0.8.1
affine==2.3.0
alabaster==0.7.12
anaconda-client==1.7.2
anaconda-navigator==1.9.7
anaconda-project==0.8.3
asn1crypto==1.2.0
astor==0.8.0
astroid==2.3.3
astropy==3.2.3
atomicwrites==1.3.0
attrs==19.3.0
Babel==2.7.0
backcall==0.1.0
backports.functools-lru-cache==1.6.1
backports.os==0.1.1
backports.shutil-get-terminal-size==1.0.0
backports.tempfile==1.0
backports.weakref==1.0.post1
basemap==1.2.0
bayesian-optimization==1.0.1
beautifulsoup4==4.8.1
bitarray==1.1.0
bkcharts==0.2
bleach==3.1.0
bokeh==1.4.0
boto==2.49.0
Bottleneck==1.3.1
branca==0.3.1
cachetools==3.1.1
Cartopy==0.17.0
certifi==2019.9.11
cffi==1.13.2
cftime==1.0.4.2
chardet==3.0.4
Click==7.0
click-plugins==1.1.1
cligj==0.5.0
cloudpickle==1.2.2
clyent==1.2.2
colorama==0.4.1
conda==4.7.12
conda-build==3.18.9
conda-package-handling==1.6.0
conda-verify==3.4.2
contextily==1.0rc2
contextlib2==0.6.0.post1
cryptography==2.8
cycler==0.10.0
Cython==0.29.14
cytoolz==0.10.1
dask==2.8.0
decorator==4.4.1
defusedxml==0.6.0
Depre

通过freeze可以看到已经安装的所有库以及其版本。可以将这些全部拷贝，然后创建一个requirements.txt，放于其中。可以检查下，把自己用不到的库去掉。

接下来看看如何根据requirements.txt安装库。

1. 打开终端
2. 进入 requirements.txt 所在的文件夹
3. 运行： pip install -r requirements.txt

以上在virtual environment中执行上述操作更好。

### pipenv

前面提到了pip和virtualenv，其实还有一个更强大的工具pipenv，本节就记录下其基本内容。本节参考了：[pipenv使用指南](https://crazygit.wiseturtles.com/2018/01/08/pipenv-tour/)，[Pipenv: A Guide to the New Python Packaging Tool](https://realpython.com/pipenv-guide/) 以及 [Pipenv——最好用的python虚拟环境和包管理工具](https://www.cnblogs.com/zingp/p/8525138.html)等。

首先，简单概述下，pipenv是**Python官方推荐的包管理工具**。可以说，它**集成了virtualenv, pip和pyenv三者的功能**。其目的旨在集合了所有的包管理工具的长处，如: npm, yarn, composer等的优点。

它能够**自动为项目创建和管理虚拟环境**，从Pipfile文件添加或删除安装的包，同时生成Pipfile.lock来锁定安装包的版本和依赖信息，避免构建错误。

pipenv主要解决了如下问题:

- 不用再单独使用**pip和virtualenv**, 现在它们**合并**在一起了
- **不用再维护requirements.txt**, 使用**Pipfile和Pipfile.lock**来代替
- 可以使用多个python版本(python2和python3)
- 在安装了pyenv的条件下，可以自动安装需要的Python版本

接下来，看看pipenv的来龙去脉。首先明确pipenv解决了什么问题？

#### Problems that Pipenv Solves

从上面说到的requirements.txt说起，比如在requirements中有flask==0.12.1，虽然指定了flask的版本，但是flask的依赖的版本是没有指定的，如果直接使用上一节说到的pip install安装的话，都会安装最新版的依赖。这可能会导致一些小问题。这就是real issue：**the build isn’t deterministic**。 即给定相同的requirements.txt文件，安装的库却可能是不同的。

比较常用的解决方法是上一节也提到过的freeze，这样就能获取所有的库及其版本号。执行之后，将结果copy到requirements.txt即可。然而，这个方法却有另外的问题。

因为，旧版本可能会有bug，有可能某个版本的依赖是需要及时更新补丁的，如果用了freeze的办法得到requirements文件，现在就需要手动修改一下某个依赖库。实际上，很多时候也并不需要一直保持在现有的版本下，很多依赖库使用最新版本可能更好更安全。这一部分的问题就是：**How do you allow for deterministic builds for your Python project without gaining the responsibility of updating versions of sub-dependencies?**

答案就是Pipenv。

现在再看另一个问题。当处理多个项目时，如果项目A需要django 1.9版本，另一个项目B需要django 1.10版本。那么这时候就需要使用前面提到的 virtual environment 来处理。工具就是venv 。现在Pipenv是包含了venv 的功能的。

然后还有一个Dependency Resolution。什么意思？加入 requirements.txt 文件中有如下代码：

```python requirements
package_a
package_b
```

假如package_a有依赖 package_c>=1.0，而package_b 有依赖 package_c<=2.0. 那么现在就是要求package_c (being >=1.0 and <=2.0) 。现在想要工具自动选择一个合适的版本，这就是“dependency resolution.”现在如果在requirements文件中加入下列语句：

```python requirements
package_c>=1.0,<=2.0
package_a
package_b
```

不过如前所述，如果package_a改变了它的requirement，那么按照指定的requirements安装可能会出错。

所以需要更加智能的安装工具，在不明确指定子依赖的情况下，能选择满足所有条件的库安装。

#### Pipenv Introduction

``` Shell
pip install pipenv
```

安装了pipenv之后，就可以忘记pip了。因为它可以完全替代pip。它还会引入两个文件，一个Pipfile，替代requirements.txt，另一个是Pipfile.lock来执行deterministic builds。

Pipenv整合了pip和virtualenv，可以让我们以简单的方式使用。

比如创建一个virtual environment，在项目的根目录下，打开命令行，然后直接使用一下语句即可：

```Shell
pipenv shell
```

该语句会为该项目创建好一个同名虚拟环境，并在该项目文件夹下创建一个Pipfile文件，然后现在的命令行下已经处在该项目的虚拟环境下了。

Pipfile是替代 requirements.txt 的。其语法是TOML的，关于TOML，可以参考：[TOML 教程 - 可能是目前最好的配置文件格式](https://zhuanlan.zhihu.com/p/50412485) ，总之是一个比较新的配置文件的格式。

如果不小心创建错了，想要删除，则可以在同一个文件夹下执行下面语句：

```Shell
pipenv --rm
```

删除之后，Pipfile还在，如果不想要的话，手动删除即可。

接下来就可以安装包了。首先可以看看虚拟环境下有什么包已经安装了：

```Shell
pipenv graph
```

上述语句可以查看所有包依赖情况。

安装package语句类似如下形式：

```Shell
pipenv install numpy
```

一旦安装了一个package, pipenv就会再生成一个Pipfile.lock文件。

Pipfile.lock是确保 deterministic builds 的，是一个JSON文件。其中有指定子依赖的版本。

如果想要卸载安装的package，使用下面语句：

```Shell
pipenv uninstall numpy
```

卸载所有：

```Shell
pipenv uninstall --all
```

如果想要安装的是pytest，并且只想让它在测试开发的时候才用，生产时候不适用，可以加上--dev参数：

```Shell
 pipenv install pytest --dev
```

如果想要将库推到生产环境下，需要锁定下安装环境：

```Shell
pipenv lock
```

这样，Pipfile.lock就不要再手动修改了。以上就是利用pipenv的一些基本操作。

查看有哪些虚拟环境，对应项目在哪，可以使用下列语句：

```Shell
pipenv --venv
pipenv --where
```

退出当前的虚拟环境直接使用：

```Shell
deactivate
```

如果想重新进入某个虚拟环境，可以用pipenv --venv找到该环境，比如/home/owen/.local/share/virtualenvs/hydrus--ORegRFb，然后使用类似如下代码即可：

```Shell
. /home/owen/.local/share/virtualenvs/hydrus--ORegRFb/bin/activate
```

如果不需要将自己的项目打包发布，那么到这就ok了。

如果需要在IDE中配置虚拟环境，比如在Pycharm中配置，可以参考官网的步骤：[Pipenv environment](https://www.jetbrains.com/help/pycharm/pipenv.html)，但是我按照官方的没成功，因为没有.local/bin文件夹，这是因为我个人安装pipenv的时候，用的是anaconda下的pip，所以pipenv是安装在ananconda的文件夹下面了，在ananconda/bin文件夹下即可看到pipenv的可执行文件。

注意首先在项目文件夹下创建虚拟环境（项目根目录下终端执行 pipenv shell），然后再到pycharm的该项目下，将pipenv添加到interpreter路径中。

如果需要发布自己的项目，即做一个可以让别人import来使用的代码包，可以使用setup.py ，一般的工作流是这样的：

- setup.py
- install_requires keyword should include whatever the package “minimally needs to run correctly.”
- Pipfile
- Represents the concrete requirements for your package
- Pull the minimally required dependencies from setup.py by installing your package using Pipenv:
    - Use pipenv install '-e .'
    - That will result in a line in your Pipfile that looks something like "e1839a8" = {path = ".", editable = true}.
- Pipfile.lock
- Details for a reproducible environment generated from pipenv lock

关于setup，以后再详细记录。。。

### Installing external services

前面说完了关于依赖包的安装，接着说说installing external services的事。

如果你存储数据到数据库中了，那么数据库很可能会用到一些服务，比如MySQL，Postgres，Redis等。比如要用Postgres，那么你就要在电脑上安装它。还要注意由于你可能使用的操作系统不同，因此安装的情况也不一样。所以，要每个情况单独处理。

安装好之后，就可以运行自己的代码程序了。如果你开发web服务端软件的话，你就知道有一些关于微服务的事，你可能有很多服务，管理这些服务还需要一些设置的，不知道就算了，不重要。

总之，就是完成Installing external services前面的事情之后，还是有很多麻烦事。这也就是为什么要使用docker的motivation之一。

## Getting Your Python App Running with Docker

Docker的运用并不是一件容易的事情，所以不存在简单一两行代码就能解决上面说的所有事情的情况。不过如果非常了解setup的东西，并且理解docker是什么，这个过程还是相对简单的。

使用docker就不需要担心项目隔离，因为Docker会为您处理这个问题。即用Python 2或3运行应用程序，每个应用程序有自己的依赖关系，这些都是是没有问题的，Virtualenv甚至不会被使用。

另外，安装外部服务也很容易。你不会直接在你的主操作系统上安装它们。它们将在您的计算机上本地运行，但是它们将通过Docker运行。例如，启动和运行Postgres或Redis需要运行一个命令，不需要高级配置。

还有最后,不需要使用很多各种devops工具,不需要担心外部服务运行时你不能开发你的应用程序,因为docker有自己的在几秒钟内用一个命令启动和关闭你所有的应用服务的工具。

不过，还是那句话，docker的运用并不容易。

首先要安装docker。

然后还要理解docker大致是如何工作的。这个并不太容易。

如果不做开发，也没有很多外部services需要使用，那么可以暂时不管它。前面说的已经足够用了。关于docker的更多内容后续再补充。